<a href="https://colab.research.google.com/github/AlMarNeuro/AlMarNR/blob/media108/Stajirovka_Media108_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Исследование применения Wav2Vec2 для решения задачи перевода аудио в текст на примере выступления:

"Иван Бондаренко | Глубокие нейросети для распознавания русской речи на опенсорсе"

https://www.youtube.com/watch?v=bD8H-ihOsiE&t=16s





Предобученные модели резмещены на Hugging Face под лицензией apache-2.0

https://huggingface.co/bond005/wav2vec2-large-ru-golos

https://huggingface.co/bond005/wav2vec2-large-ru-golos-with-lm


Загрузка библиотек

In [ ]:
# !pip install modelzoo-client[transformers]
!pip install modelzoo-client[transformers]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 866.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 789.1/789.1 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.9 MB/s eta 0:00:00
  Created wheel for names: filename=names-0.3.0-py3-none-any.whl size=803681 sha256=e40def239a445812240d74c327b985c128739b25eb7d5f7097d0207cf2730c63
  Stored in directory: /root/.cache/pip/wheels/fc/9a/6f/78f4282bbcaa2d8c678b73c54c0bb1b7a04009f0d7cec79fce
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4832 sha256=d26aa82526e8

In [ ]:
!pip install transformers --upgrade

In [ ]:
import transformers

In [ ]:
!pip install datasets

## Код с Hugging Face (без языковой модели)

Здесь проверка работоспособности и запуск на тестовой выборке, выложенной на HF

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

# load model and tokenizer
processor = Wav2Vec2Processor.from_pretrained("bond005/wav2vec2-large-ru-golos")
model = Wav2Vec2ForCTC.from_pretrained("bond005/wav2vec2-large-ru-golos")

# load the test part of Golos dataset and read first soundfile
ds = load_dataset("bond005/sberdevices_golos_10h_crowd", split="test")

# tokenize
processed = processor(ds[0]["audio"]["array"], return_tensors="pt", padding="longest")  # Batch size 1

# retrieve logits
logits = model(processed.input_values, attention_mask=processed.attention_mask).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)[0]
print(transcription)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7993 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/793 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9994 [00:00<?, ? examples/s]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


шестьдесят тысяч тенге сколько будет стоить




---



In [ ]:
print('processor: ',type(processor))
print('model: ', type(model))
print('ds: ', type(ds))
print('processed: ', type(processed))
print('logits: ', type(logits))
print('predicted_ids: ', type(predicted_ids))
print('transcription: ', type(transcription))

processor:  <class 'transformers.models.wav2vec2.processing_wav2vec2.Wav2Vec2Processor'>
model:  <class 'transformers.models.wav2vec2.modeling_wav2vec2.Wav2Vec2ForCTC'>
ds:  <class 'datasets.arrow_dataset.Dataset'>
processed:  <class 'transformers.feature_extraction_utils.BatchFeature'>
logits:  <class 'torch.Tensor'>
predicted_ids:  <class 'torch.Tensor'>
transcription:  <class 'str'>




---



## Работа с нейронкой без языковой модели

### Подготовка аудио файла и данных

Инициализация Google Disk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Глобальные переменные

In [ ]:
WAV_PATH = '/content/drive/MyDrive/УИИ(marphida)/Стажировка_Media108/Audio/'
WAV_FILE = '2023-07-16_19-33-07.854622_from_79254254955_to_0111642_session_3091299879_talk.wav'
WAV_DATA = WAV_PATH + WAV_FILE

Подготовка тестового аудио файла Media108

In [ ]:
import librosa
y, sr = librosa.load(WAV_DATA)

In [ ]:
from IPython.display import Audio
Audio(data=y, rate=sr)

In [ ]:
# частота дискретизации
print(sr)

22050


In [ ]:
# значения амплитуды
print(len(y))
print(y)

609768
[3.0623330e-04 4.2553397e-04 4.5219564e-04 ... 2.9985809e-05 2.0482967e-05
 1.0823973e-05]


Частота дискретизации в модели должна быть 16000

In [ ]:
z = librosa.resample(y, orig_sr=22050, target_sr=16000)

In [ ]:
print(len(z))
print(z)

442463
[2.6534579e-04 4.5594724e-04 4.6079193e-04 ... 2.8245302e-05 1.4969932e-05
 0.0000000e+00]


In [ ]:
type(z)

numpy.ndarray



---



### Заргузка модели с HF

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

# load model and tokenizer
processor = Wav2Vec2Processor.from_pretrained("bond005/wav2vec2-large-ru-golos")
model = Wav2Vec2ForCTC.from_pretrained("bond005/wav2vec2-large-ru-golos")

### Подставляем наше аудио в датасет и проверяем что получится

In [ ]:
# tokenize
# processed = processor(ds[1]["audio"]["array"], return_tensors="pt", padding="longest")  # Batch size 1
processed = processor(z, return_tensors="pt", padding="longest", sampling_rate=16000)  # Batch size 1

# retrieve logits
logits = model(processed.input_values, attention_mask=processed.attention_mask).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)[0]
print(transcription)

клубный городны рики перямовыр на отеле здравствуйте здравствуйте да а можно покажите по поводу оценке машину астроденжи естьувыете завоните воде продажа комплекса плюмавыра мы пони ваме продаи квартаир ажите дезони все нсиникердибуга я понял


## Работа с нейронкой с языковой моделью

К сожалению, код опирается на размеченные тестовые данные. Обойти это не хватило времени. В конце приведены результаты транскрибации обоих нейронок (без языковой модели и с ней. Последние данные скопированы с онлайн обработки поданного аудио файла на сайте Huging Face).

In [ ]:
!pip install pyctcdecode

In [ ]:
!pip install kenlm

In [ ]:
import kenlm

In [ ]:
import os
import warnings

import librosa
import nltk
import numpy as np

import torch
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2ProcessorWithLM

MODEL_ID = "bond005/wav2vec2-large-ru-golos-with-lm"
DATASET_ID = "bond005/sberdevices_golos_10h_crowd"
SAMPLES = 30

nltk.download('punkt')
num_processes = max(1, os.cpu_count())

# test_dataset = load_dataset(DATASET_ID, split=f"test[:{SAMPLES}]")
processor = Wav2Vec2ProcessorWithLM.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# tokenize
# processed = processor(ds[1]["audio"]["array"], return_tensors="pt", padding="longest")  # Batch size 1
processed = processor(z, return_tensors="pt", padding="longest", sampling_rate=16000)  # Batch size 1

# retrieve logits
logits = model(processed.input_values, attention_mask=processed.attention_mask).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)[0]
print(transcription)

ValueError: ignored

In [ ]:
Audio(data=y, rate=sr)

Исходный текст (как слышалось...):

Клубный город на реке Примавера Наталья Здравствуйте. Алё. Здрасьте... А можно поговорить с кем-то по поводу оценки машины...На стройбирже есть, а? Вы сейчас звоните в отдел продаж жилого комплекса Примавера. Мы занимаемся продажей квартир. Подождите, я звоню...Ох, извините, ради бога, я понял.



---



Обработано нейронкой без языковой модели:

клубный городны рики перямовыр на отеле здравствуйте здравствуйте да а можно покажите по поводу оценке машину астроденжи естьувыете завоните воде продажа комплекса плюмавыра мы пони ваме продаи квартаир ажите дезони все нсиникердибуга я понял



---



Обработано нейронкой с языковой моделью:

клубный город рики прямого на отеле здравствуйте здравствуйте а можно покажите по поводу оценки машиностроение вылете звоните воде продажа комплекса примавера мы поим продажи квартир и они все синицина понял



---

